In [1]:
from ortools.linear_solver import pywraplp
from urllib.request import urlopen
import numpy as np
import json

In [2]:
def fetch_json_data(url):
    """fetches JSON data from a url
    """
    return json.loads(urlopen(url).read())

In [3]:
# fetch the roasters in the format
# IdPilot, IdCoPilot, IdNavigator
# we use different job classes:
# 0: Pilot
# 1: Copilot
# 2: IdNavigator
# for each job class the different employees marked with ids
# note that ids from different job classes are different persons
# i.e. pilot 4 is not the same person as copilot 4
possible_roasters = fetch_json_data('https://raw.githubusercontent.com/eineruntervielen/vl-optimierung-daten/master/airline_crew_data.json')

In [4]:
possible_roasters

{'roster_0': [3, 18, 30],
 'roster_1': [4, 4, 36],
 'roster_2': [1, 5, 9],
 'roster_3': [7, 17, 30],
 'roster_4': [10, 23, 23],
 'roster_5': [8, 10, 25],
 'roster_6': [19, 29, 36],
 'roster_7': [3, 4, 17],
 'roster_8': [19, 28, 40],
 'roster_9': [11, 24, 31],
 'roster_10': [18, 30, 33],
 'roster_11': [22, 25, 26],
 'roster_12': [13, 15, 26],
 'roster_13': [21, 27, 28],
 'roster_14': [7, 12, 33]}

## optimization task

Maximize the number of roasters used with the constraint that each person can only be present in one roaster at a time.

In [5]:
# define the solver
solver = pywraplp.Solver('RoasterSolver', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [6]:
# define decision variables
# for each roaster there's one decision variable (a bool) whether the
# roaster at the given index shall be used or not
decision_vars = [solver.IntVar(0, 1, roaster) for roaster in possible_roasters]

In [7]:
# create for each job class
# a list of all employees
all_employees = list(zip(*possible_roasters.values()))

In [15]:
# define constraints
num_constraints = 0
# loop through all three job classes
for job_class in range(len(all_employees)):
    # since the goal is to maximize the number of roasters,
    # and the contstrant is that one person can only be present
    # in one roaster at a time
    # build the constrains by looping through
    # all job classes and then finding
    # for each employee if he/she is represented in mulitple roasters
    # if so, add the contraints which roasters can only be used at the same time
    # (because one person would be in multiple if they were both used)
    # i.e.
    # if pilot 4 is in roaster 10 and 12 -->
    # decision_vars[9] + decision_vars[11] <= 1
    
    
    # loop through each employee of the job class once
    for current_employee in np.unique(all_employees[job_class]):
        # list of (overlapping) roasters 
        # in form of decision variables for current employee
        vars_list = []
        
        # find occurences of current employee in all possible roasters
        # and add vars to list
        for roaster_name, x in zip(possible_roasters, decision_vars):  # x as decision var
            if possible_roasters[roaster_name][job_class] == current_employee:
                vars_list.append(x)
    
        # every employee can only be included in one roaster
        # add the contraint only if it matters (if the employee is only represented
        # in 0 or 1 roasters there's no contraint for him/her)
        if len(vars_list) > 1:
            num_constraints += 1
            solver.Add(sum(vars_list) <= 1)

print(f'{num_constraints} constraints created')

8 constraints created


In [16]:
# define goal:
# maximize the number of roasters used
solver.Maximize(sum(decision_vars))

In [17]:
# solve the MIP
status = solver.Solve()

In [18]:
# dislay solution
if status == pywraplp.Solver.OPTIMAL:
    print('\nSolution:')
    print('Objective value =', solver.Objective().Value())
    print('Choosen Roasters:')
    
    for decisionVar in decision_vars:
        if decisionVar.solution_value():
            print(decisionVar)
else:
    print('The problem does not have an optimal solution.')



Solution:
Objective value = 10.0
Choosen Roasters:
roster_0
roster_1
roster_2
roster_4
roster_5
roster_8
roster_9
roster_10
roster_11
roster_13
